In [2]:
import pandas as pd

def gerar_relatorio_diferenca(local_path, gcp_path, output_path, output_missing_path):
    # Carregar os dados das bases CSV
    local_df = pd.read_csv(local_path, sep=";")
    gcp_df = pd.read_csv(gcp_path)

    # Remover duplicatas em ambas as bases com base no ID
    local_df = local_df.drop_duplicates(subset='ID')
    gcp_df = gcp_df.drop_duplicates(subset='ID')

    # Realizar a junção completa (FULL OUTER JOIN) com indicador de origem das linhas
    merged_df = pd.merge(local_df, gcp_df, on='ID', how='outer', suffixes=('_local', '_gcp'), indicator=True)

    # Trazer apenas as linhas que estão em uma tabela mas não na outra
    missing_rows = merged_df[merged_df['_merge'] != 'both']

    # Exportar essas linhas para um arquivo separado
    missing_rows.to_csv(output_missing_path, index=False)

    # Função para gerar validação para cada coluna
    def validar_coluna(coluna_local, coluna_gcp):
        if pd.isna(coluna_local) or pd.isna(coluna_gcp):
            return 'valor nulo'
        elif coluna_local != coluna_gcp:
            return 'não ok'
        else:
            return 'ok'

    # Criar colunas de validação para cada campo
    merged_df['Validacao_CODE_GENDER'] = merged_df.apply(lambda row: validar_coluna(row['CODE_GENDER_local'], row['CODE_GENDER_gcp']), axis=1)
    merged_df['Validacao_FLAG_OWN_CAR'] = merged_df.apply(lambda row: validar_coluna(row['FLAG_OWN_CAR_local'], row['FLAG_OWN_CAR_gcp']), axis=1)
    merged_df['Validacao_FLAG_OWN_REALTY'] = merged_df.apply(lambda row: validar_coluna(row['FLAG_OWN_REALTY_local'], row['FLAG_OWN_REALTY_gcp']), axis=1)
    merged_df['Validacao_CNT_CHILDREN'] = merged_df.apply(lambda row: validar_coluna(row['CNT_CHILDREN_local'], row['CNT_CHILDREN_gcp']), axis=1)
    merged_df['Validacao_AMT_INCOME_TOTAL'] = merged_df.apply(lambda row: validar_coluna(row['AMT_INCOME_TOTAL_local'], row['AMT_INCOME_TOTAL_gcp']), axis=1)
    merged_df['Validacao_NAME_INCOME_TYPE'] = merged_df.apply(lambda row: validar_coluna(row['NAME_INCOME_TYPE_local'], row['NAME_INCOME_TYPE_gcp']), axis=1)
    merged_df['Validacao_NAME_EDUCATION_TYPE'] = merged_df.apply(lambda row: validar_coluna(row['NAME_EDUCATION_TYPE_local'], row['NAME_EDUCATION_TYPE_gcp']), axis=1)
    merged_df['Validacao_NAME_FAMILY_STATUS'] = merged_df.apply(lambda row: validar_coluna(row['NAME_FAMILY_STATUS_local'], row['NAME_FAMILY_STATUS_gcp']), axis=1)
    merged_df['Validacao_NAME_HOUSING_TYPE'] = merged_df.apply(lambda row: validar_coluna(row['NAME_HOUSING_TYPE_local'], row['NAME_HOUSING_TYPE_gcp']), axis=1)
    merged_df['Validacao_DAYS_BIRTH'] = merged_df.apply(lambda row: validar_coluna(row['DAYS_BIRTH_local'], row['DAYS_BIRTH_gcp']), axis=1)
    merged_df['Validacao_DAYS_EMPLOYED'] = merged_df.apply(lambda row: validar_coluna(row['DAYS_EMPLOYED_local'], row['DAYS_EMPLOYED_gcp']), axis=1)
    merged_df['Validacao_FLAG_MOBIL'] = merged_df.apply(lambda row: validar_coluna(row['FLAG_MOBIL_local'], row['FLAG_MOBIL_gcp']), axis=1)
    merged_df['Validacao_FLAG_WORK_PHONE'] = merged_df.apply(lambda row: validar_coluna(row['FLAG_WORK_PHONE_local'], row['FLAG_WORK_PHONE_gcp']), axis=1)
    merged_df['Validacao_FLAG_PHONE'] = merged_df.apply(lambda row: validar_coluna(row['FLAG_PHONE_local'], row['FLAG_PHONE_gcp']), axis=1)
    merged_df['Validacao_FLAG_EMAIL'] = merged_df.apply(lambda row: validar_coluna(row['FLAG_EMAIL_local'], row['FLAG_EMAIL_gcp']), axis=1)
    merged_df['Validacao_OCCUPATION_TYPE'] = merged_df.apply(lambda row: validar_coluna(row['OCCUPATION_TYPE_local'], row['OCCUPATION_TYPE_gcp']), axis=1)
    merged_df['Validacao_CNT_FAM_MEMBERS'] = merged_df.apply(lambda row: validar_coluna(row['CNT_FAM_MEMBERS_local'], row['CNT_FAM_MEMBERS_gcp']), axis=1)

    # Criar resumo das diferenças por coluna
    resumo_diferencas = {}
    colunas_validacao = [col for col in merged_df.columns if col.startswith('Validacao_')]

    for coluna in colunas_validacao:
        resumo_diferencas[coluna] = (merged_df[coluna] == 'não ok').sum()

    # Contar as linhas que estão em uma tabela mas não na outra
    linhas_faltantes = merged_df['_merge'].value_counts().to_dict()
    resumo_diferencas['Linhas apenas em local_df'] = linhas_faltantes.get('left_only', 0)
    resumo_diferencas['Linhas apenas em gcp_df'] = linhas_faltantes.get('right_only', 0)

    # Exportar as diferenças encontradas
    merged_df.to_csv(output_path, index=False)

    return resumo_diferencas

# Uso da função
local_path = 'application_record_local.csv'
gcp_path = 'application_record_gcp.csv'
output_path = 'diferencas_local_vs_gcp.csv'
output_missing_path = 'linhas_faltantes_local_ou_gcp.csv'

resumo_diferenca = gerar_relatorio_diferenca(local_path, gcp_path, output_path, output_missing_path)
print("Resumo das diferenças por coluna:")
for coluna, total in resumo_diferenca.items():
    print(f"{coluna}: {total}")

print(f"Relatório de diferenças gerado: {output_path}")
print(f"Linhas faltantes entre as bases gerado: {output_missing_path}")


Resumo das diferenças por coluna:
Validacao_CODE_GENDER: 434459
Validacao_FLAG_OWN_CAR: 0
Validacao_FLAG_OWN_REALTY: 0
Validacao_CNT_CHILDREN: 0
Validacao_AMT_INCOME_TOTAL: 434459
Validacao_NAME_INCOME_TYPE: 0
Validacao_NAME_EDUCATION_TYPE: 0
Validacao_NAME_FAMILY_STATUS: 0
Validacao_NAME_HOUSING_TYPE: 0
Validacao_DAYS_BIRTH: 434459
Validacao_DAYS_EMPLOYED: 0
Validacao_FLAG_MOBIL: 0
Validacao_FLAG_WORK_PHONE: 0
Validacao_FLAG_PHONE: 0
Validacao_FLAG_EMAIL: 0
Validacao_OCCUPATION_TYPE: 0
Validacao_CNT_FAM_MEMBERS: 0
Linhas apenas em local_df: 4051
Linhas apenas em gcp_df: 0
Relatório de diferenças gerado: diferencas_local_vs_gcp.csv
Linhas faltantes entre as bases gerado: linhas_faltantes_local_ou_gcp.csv
